In [1]:
from gensim.models import Word2Vec
from utils import *

train_data = read_train_data()
test_data = read_test_data()

model = Word2Vec(sentences=[['1', '2', '3'], ['2', '3', '4']], vector_size=100, window=5, min_count=1, workers=4)
model.wv.most_similar('1', topn=10)


[('2', 0.06797593086957932),
 ('4', -0.013514931313693523),
 ('3', -0.1116705983877182)]

In [2]:
model = Word2Vec(sentences=list(train_data['content']), vector_size=100, window=5, min_count=1, workers=12)

In [3]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import numpy as np

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, output_size, kernel_sizes=[3, 4, 5], num_filters=64):
        super(TextCNN, self).__init__()

        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False

        # 卷积层
        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters, (k, embed_size)) for k in kernel_sizes])

        # 全连接层
        self.fc = nn.Linear(num_filters * len(kernel_sizes), output_size)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1) # [batch, 1, seq_len, embed_size]
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # [batch, num_filters, seq_len-k+1]
        x = [F.max_pool1d(line, line.size(2)).squeeze(2) for line in x] # [batch, num_filters]
        x = torch.cat(x, 1) # [batch, num_filters * len(kernel_sizes)]
        logits = self.fc(x)
        return logits


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import numpy as np

# 创建词汇表
vocab = {word: index for index, word in enumerate(model.wv.index_to_key)}
vocab_size = len(vocab)

# 创建嵌入矩阵
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in vocab.items():
    embedding_matrix[index] = model.wv[word]

# 文本转换为索引
def text_to_index(texts):
    indices = []
    for text in texts:
        index_list = [vocab[word] for word in text if word in vocab]
        indices.append(index_list)
    return indices

# 对文本和标签进行处理
x_train = text_to_index(train_data['content'])
y_train = train_data['label']



# pad sequences
def pad_sequences(sequences, maxlen=None, padding='pre', truncating='pre', value=0.):

    lengths = [len(s) for s in sequences]
    nb_samples = len(sequences)

    if maxlen is None:
        maxlen = np.max(lengths)

    x = (np.ones((nb_samples, maxlen)) * value).astype(np.int32)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError("Truncating type '%s' not understood" % padding)

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError("Padding type '%s' not understood" % padding)

    return x

x_train = pad_sequences(x_train, maxlen=214, padding='post', truncating='post')


x_train_tensor = torch.tensor(x_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm


output_size = len(set(y_train))
embed_size = 100
patience = 5

# Use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=5, shuffle=True, random_state=42)

acc_list = []
for fold, (train_index, val_index) in enumerate(kf.split(x_train_tensor)):
    print(f"Fold {fold + 1}")
    x_train_fold, x_val_fold = x_train_tensor[train_index], x_train_tensor[val_index]
    y_train_fold, y_val_fold = y_train_tensor[train_index], y_train_tensor[val_index]

    train_dataset = TensorDataset(x_train_fold, y_train_fold)
    val_dataset = TensorDataset(x_val_fold, y_val_fold)

    train_loader = DataLoader(train_dataset, batch_size=128)
    val_loader = DataLoader(val_dataset, batch_size=128)

    # 定义和训练模型
    model = TextCNN(vocab_size, embed_size, output_size).to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in tqdm(range(100)): # 使用tqdm库进行进度展示
        model.train()
        for batch_text, batch_labels in train_loader:
            # 将数据放到GPU上
            batch_text, batch_labels = batch_text.to(device), batch_labels.to(device) 
            optimizer.zero_grad()
            logits = model(batch_text)
            loss = loss_function(logits, batch_labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_text, batch_labels in val_loader:
                batch_text, batch_labels = batch_text.to(device), batch_labels.to(device)
                logits = model(batch_text)
                loss = loss_function(logits, batch_labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break

    # 验证模型
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_text, batch_labels in val_loader:
            batch_text, batch_labels = batch_text.to(device), batch_labels.to(device)
            logits = model(batch_text)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

    # 使用classification_report输出结果, 小数点后四位
    print(classification_report(all_labels, all_preds, digits=4))
    acc = accuracy_score(all_labels, all_preds)
    acc_list.append(acc)
    
# print mean accuracy
print(f"Mean accuracy: {np.mean(acc_list)}")

Fold 1


  0%|          | 0/100 [00:00<?, ?it/s]

  9%|▉         | 9/100 [00:28<04:44,  3.12s/it]


Early stopping at epoch 10
              precision    recall  f1-score   support

           0     0.9858    0.9979    0.9918      2357
           1     0.9879    0.9233    0.9545       443

    accuracy                         0.9861      2800
   macro avg     0.9868    0.9606    0.9731      2800
weighted avg     0.9861    0.9861    0.9859      2800

Fold 2


 15%|█▌        | 15/100 [00:45<04:16,  3.02s/it]


...

Early stopping at epoch 21
              precision    recall  f1-score   support

           0     0.9928    0.9957    0.9943      2348
           1     0.9775    0.9624    0.9699       452

    accuracy                         0.9904      2800
   macro avg     0.9852    0.9791    0.9821      2800
weighted avg     0.9903    0.9904    0.9903      2800

Fold 4


 20%|██        | 20/100 [01:00<04:02,  3.03s/it]


Early stopping at epoch 21
              precision    recall  f1-score   support

           0     0.9920    0.9970    0.9945      2362
           1     0.9836    0.9566    0.9699       438

    accuracy                         0.9907      2800
   macro avg     0.9878    0.9768    0.9822      2800
weighted avg     0.9907    0.9907    0.9907      2800

Fold 5


 14%|█▍        | 14/100 [00:43<04:25,  3.09s/it]

Early stopping at epoch 15
              precision    recall  f1-score   support

           0     0.9958    0.9937    0.9947      2373
           1     0.9653    0.9766    0.9709       427

    accuracy                         0.9911      2800
   macro avg     0.9805    0.9851    0.9828      2800
weighted avg     0.9911    0.9911    0.9911      2800

Mean accuracy: 0.9900714285714285


In [5]:
# 设定超参数
output_size = len(set(y_train))
embed_size = 100
patience = 5

# Use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 创建数据加载器

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128)

# 实例化模型
model = TextCNN(vocab_size, embed_size, output_size).to(device)

# 损失函数和优化器
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
for epoch in range(10):
    for batch_text, batch_labels in train_loader:
        optimizer.zero_grad()
        logits = model(batch_text.to(device))
        loss = loss_function(logits, batch_labels.to(device))
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch} | Loss: {loss.item()}')


Epoch 0 | Loss: 0.07313285768032074
Epoch 1 | Loss: 0.008089765906333923
Epoch 2 | Loss: 0.0057031516917049885
Epoch 3 | Loss: 0.004010699689388275
Epoch 4 | Loss: 0.0008793951128609478
Epoch 5 | Loss: 0.0005976419779472053
Epoch 6 | Loss: 0.00046915956772863865
Epoch 7 | Loss: 0.0003809006593655795
Epoch 8 | Loss: 0.00029977751546539366
Epoch 9 | Loss: 0.00024083467724267393


In [6]:
x_test = text_to_index(test_data['content'])
x_test = pad_sequences(x_test, maxlen=214, padding='post', truncating='post')
x_test_tensor = torch.tensor(x_test, dtype=torch.long).to(device)

test_dataset = TensorDataset(x_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

model.eval()
test_pred = []
with torch.no_grad():
    for x_batch in test_loader:
        x_batch = x_batch[0].to(device)
        logits = model(x_batch)
        y_pred = torch.argmax(logits, dim=1)
        test_pred.extend(y_pred.cpu().numpy())

test_data['label'] = test_pred
test_data[['name', 'label']].to_csv('textcnn.csv', index=False)